# Face Mask Detection

## 1. Preparing the Environment
- Install TF = 2.2.0
- Clone the tensorflow mdoels repo
- compiling protocol bus buffer for tf record
- Execturing the setup test

In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models
!pip -q install Cython contextlib2 pillow lxml jupyter matplotlib

In [ ]:
import os
os.chdir("models/research")

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
import os
os.chdir('/content/models/research')

In [ ]:
#test whether builders are running properly
!python object_detection/builders/model_builder_test.py

2020-09-30 09:51:04.417400: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


## 2. Data Preparation

### 2.1 Download Data

In [ ]:
!wget -q https://www.dropbox.com/s/09dr3h6ebei8fhf/mask_detection_dataset.zip

In [ ]:
!unzip mask_detection_dataset.zip

In [ ]:
os.getcwd()
os.chdir("/content/models/research")

### 2.2 Move the face mask images and labels to a common folder

In [ ]:
%mkdir training_demo
%cd training_demo/
%mkdir images
%cd images
%mkdir train
%mkdir test
%mkdir data
os.chdir("/content/models/research")
print(os.getcwd())
!cp -R annotations/. training_demo/images/data/
!cp -R images/. training_demo/images/data/

/content/models/research/training_demo
/content/models/research/training_demo/images
/content/models/research


In [ ]:
os.getcwd()

'/content/models/research'

In [ ]:
# Getting the list of images and labels
label_list = os.listdir("annotations/")
img_list = os.listdir("images/")
print(label_list[:5])
print(img_list[:5])
print(len(img_list))
print(len(label_list))

['maksssksksss755.xml', 'maksssksksss815.xml', 'maksssksksss620.xml', 'maksssksksss176.xml', 'maksssksksss836.xml']
['maksssksksss836.png', 'maksssksksss344.png', 'maksssksksss498.png', 'maksssksksss354.png', 'maksssksksss65.png']
853
853


### 2.3 Splitting images and labels into train and test folders

In [ ]:

import os
import shutil
import numpy as np
for img_name in img_list:
  print(img_name)
  label_name = img_name[:-4]+".xml"
  print(label_name)
  if np.random.rand(1) < 0.20:
    shutil.move("images/" + img_name, "/content/models/research/training_demo/images/test" + '/'+ img_name)
    shutil.move("annotations/" + label_name, "/content/models/research/training_demo/images/test" + '/'+ label_name)
  else:
    shutil.move("images/" + img_name, "/content/models/research/training_demo/images/train" + '/'+ img_name)
    shutil.move("annotations/" + label_name, "/content/models/research/training_demo/images/train" + '/'+ label_name)

print(len(os.listdir("/content/models/research/training_demo/images/train")))
print(len(os.listdir("/content/models/research/training_demo/images/test")))


# Downloading All utility files

In [ ]:
# Getting the label map
!wget -q https://raw.githubusercontent.com/anshupandey/Computer-Vision/master/object%20detection/mask_detection/label_map.pbtxt

In [ ]:
# downloading the tf record


In [ ]:
# downloading pipeline.config


# Generating TFrecords

### 4.1 Generating train record

In [ ]:
# Create train record
!python generate_tfrecord.py -x /content/models/research/training_demo/images/train -l /content/models/research/label_map.pbtxt -o /content/models/research/training_demo/train.record

shape of labels  (3269, 8)
['with_mask' 'without_mask' 'mask_weared_incorrect']  are the unique classes
Successfully created the TFRecord file: /content/models/research/training_demo/train.record


### 4.2 Generating test record

In [ ]:
# Create tet data record:
!python generate_tfrecord.py -x /content/models/research/training_demo/images/test -l /content/models/research/label_map.pbtxt -o /content/models/research/training_demo/test.record


# Download SSD Mobilenet model 

In [ ]:
os.getcwd()

'/content/models/research'

In [ ]:
import requests
import os
import shutil

# The script
curr_path = os.getcwd()
models_path = os.path.join(curr_path,"training_demo")
# downloading from: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'

if os.path.exists(os.path.join(models_path,"ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz")) == False:
    response = requests.get(url, stream=True)
    with open(os.path.join(models_path,"ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz"), 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response


import tarfile
filePath = os.path.join(models_path,"ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz")
os.chdir(models_path)


if (filePath.endswith("tar.gz")):
    tar = tarfile.open(filePath, "r:gz")
    tar.extractall()
    tar.close()
elif (filePath.endswith("tar")):
    tar = tarfile.open(filePath, "r:")
    tar.extractall()
    tar.close()


print("done")

done


In [ ]:
print(os.getcwd())
os.chdir("/content/models/research")
print(os.getcwd())

/content/models/research/training_demo
/content/models/research


In [ ]:
# creating a folder to store the trained check points
os.mkdir("model_output")

In [ ]:
!python object_detection/model_main_tf2.py --model_dir=model_output --pipeline_config_path=pipeline.config

2020-09-30 10:00:20.630490: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-30 10:00:22.504193: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-30 10:00:22.539661: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-30 10:00:22.540201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-09-30 10:00:22.540233: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-30 10:00:22.542047: I tensorflow/stream_executor/platform/default

In [ ]:
os.mkdir("my_model")

In [ ]:
!python object_detection/exporter_main_v2.py --input_type image_tensor --pipeline_config_path pipeline.config --trained_checkpoint_dir /content/models/research/model_output --output_directory my_model

2020-09-29 12:10:48.351892: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 12:10:50.662750: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-29 12:10:50.686759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-29 12:10:50.687504: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-09-29 12:10:50.687553: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 12:10:50.691695: I tensorflow/stream_executor/platform/defa

In [ ]:
!python object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v1_pascal.config --trained_checkpoint_prefix model_output/model.ckpt-2624 --output_directory model1/

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## 7. Test the model

In [ ]:
#Upload test file
from google.colab import files
from os import path

uploaded = files.upload()
  
for name, data in uploaded.items():
  with open('group_1.jpg', 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)

Saving IMG_20190801_173244.jpg to IMG_20190801_173244.jpg
saved file IMG_20190801_173244.jpg


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

  

  
  
# This is needed to display the images.
%matplotlib inline




from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util




# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = "my_model/saved_model/saved_model.pb"

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('', 'label_map.pbtxt')

NUM_CLASSES = 3




detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    
    
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)




def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '.'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'group_{}.jpg'.format(i)) for i in range(1, 2) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict




for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=6)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)


AttributeError: ignored

In [ ]:
output_dict

{'detection_boxes': array([[2.09379703e-01, 3.43391746e-01, 7.58175969e-01, 7.10473299e-01],
        [1.90511137e-01, 3.38057071e-01, 7.93640256e-01, 7.07008958e-01],
        [1.28045976e-01, 3.04313004e-02, 9.93710339e-01, 5.49713850e-01],
        [7.29258239e-01, 3.68917793e-01, 7.99769223e-01, 4.47305471e-01],
        [5.83610535e-02, 1.24013424e-01, 1.00000000e+00, 8.96166325e-01],
        [0.00000000e+00, 2.22693056e-01, 1.00000000e+00, 7.40810513e-01],
        [2.27501482e-01, 0.00000000e+00, 7.57023215e-01, 1.00000000e+00],
        [4.64119494e-01, 3.44192684e-02, 9.76014197e-01, 7.08901286e-01],
        [7.77669549e-01, 3.67128789e-01, 8.39256883e-01, 4.45799112e-01],
        [3.18967223e-01, 2.24059761e-01, 7.04992652e-01, 8.57137620e-01],
        [7.29109168e-01, 4.26100999e-01, 7.86178112e-01, 5.05729139e-01],
        [0.00000000e+00, 2.22693056e-01, 1.00000000e+00, 7.40810513e-01],
        [2.27501482e-01, 0.00000000e+00, 7.57023215e-01, 1.00000000e+00],
        [1.43192112